<a href="https://colab.research.google.com/github/hopperrr/segment-anything/blob/main/image_collection_alpha_transarency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/facebookresearch/segment-anything.git

# Automatically generating object masks with SAM

In [3]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
    
    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
        
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

## Set-up

In [85]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from pathlib import Path
imagepath = '/content/images/'
imagename = 'aerial2.png'
imagename_no_extension = imagename.rsplit( ".", 1 )[ 0 ]
outputfullpath = '/content/output/' + imagename
outputsmall = '/content/output/' + imagename_no_extension + '/smalls'
imagefullpath= imagepath + imagename
metapath = '/content/output/' + imagename_no_extension
image = cv2.imread(imagefullpath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
!python /content/segment-anything/scripts/amg.py --model-type "vit_h" --checkpoint /content/sam_vit_h_4b8939.pth --input /content/images/aerial2.png --output /content/output

In [87]:
df = pd.read_csv(metapath +"/metadata.csv")
image = cv2.imread(imagefullpath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
filecount = len(df)
Path(outputsmall).mkdir(parents=True, exist_ok=True) 
for index, row in df.iterrows():
  itemnum = row['id']

  masker = cv2.imread(metapath + '/' + str(int(itemnum)) + '.png')
  masker_result = cv2.bitwise_and(image, masker)
  details = df.loc[df['id'] == itemnum]
  x = int(details.iloc[0]['bbox_x0'])
  y = int(details.iloc[0]['bbox_y0'])
  x1 = int(x + details.iloc[0]['bbox_w'])
  y1 = int(y + details.iloc[0]['bbox_h'])
  cropped = masker_result[y:y1, x:x1]
  alpha = np.sum(cropped, axis=-1) > 0
  alpha = np.uint8(alpha * 255)
  res = np.dstack((cropped, alpha))
  res = cv2.cvtColor(res, cv2.COLOR_BGRA2RGBA)
  cv2.imwrite(outputsmall + '/' + str(int(itemnum)) + '.png', res)

#plt.figure(figsize=(20,20))
#plt.imshow(res)
#plt.axis('off')
#plt.show()

In [ ]:
for index, row in df.iterrows():
    print(row['bbox_x0'])

In [56]:
import shutil
shutil.rmtree(outputsmall)

In [11]:
details

,id,area,bbox_x0,bbox_y0,bbox_w,bbox_h,point_input_x,point_input_y,predicted_iou,stability_score,crop_box_x0,crop_box_y0,crop_box_w,crop_box_h
20,20,9544,0,467,84,165,60.28125,552.6875,0.98307,0.961404,0,0,1286,956


In [ ]:
!zip -r /content/file4.zip /content/output/aerial2/smalls


In [90]:
from google.colab import files
files.download("/content/file4.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>